In [9]:

import mne
import numpy as np
import matplotlib as plt
import pandas as pd
import os
import scipy.io

In [10]:
# define path
global project_path, eeg_path, data_path, label_path, post_fix, sampling
project_path = '/Volumes/CLPS_Watanabe_Lab/users/Zhiyan/SleepDecoding/'
eeg_path = 'Experiment/sbj/'
data_path = 'EEG/raw/'
label_path = 'label/'
post_fix = '_reref_01100.vhdr'
subjects = ['sldcd01', 'sldcd03', 'sldcd05','sldcd06']
naps = ['nap2'] #%, 'nap3']
sampling = 500 # hz



In [11]:
# define eeg parameters 
len_epochs = 30 # [s]
num_ch = 36

In [12]:
def load_label_raw(SUB, NAP):
    fn_path = project_path +  eeg_path + SUB + '/' + NAP + '/' +  label_path +  'stageMat_' + SUB + '_' + NAP 
    label = scipy.io.loadmat(fn_path)
    label = label['stageMat']
    return label

In [13]:
def load_bad_ch(SUB, NAP):
    fn_path = project_path +  eeg_path + SUB + '/' + NAP + '/' +  label_path + SUB + '_' + NAP + '_bad' + '.txt'
    bad = pd.read_csv(fn_path, sep=" ", header = None)
    return set(bad.iloc[:,0])


In [16]:
def load_parse_eeg(SUB, NAP):
    fn_path = project_path +  eeg_path + SUB + '/' + NAP + '/' +  data_path + SUB + NAP + post_fix
    raw = mne.io.read_raw_brainvision(fn_path, eog = ('HEOG', 'VEOG'), preload=True)
    eeg_pick = raw.copy().pick_types(eeg = True, eog = True, misc = True)
    # pick channels 
    eog_pick = raw.copy().pick_types(eeg = False, eog = True)
    emg_pick =raw.copy().pick_types(eeg = False, eog = False, misc = True)
    return eeg_pick, eog_pick, emg_pick

In [17]:
# start the loop for processing participants 
# for SUB in subjects:
#     for NAP in naps:
data_final = pd.DataFrame(columns=['data', 'channel_name', 'label', 'subject'])
for SUB in subjects:
    for NAP in naps:
    #SUB = 'sldcd01'
    #NAP = 'nap2'
        print(SUB + '_' + NAP)
    
        eeg_s, eog_s, emg_s= load_parse_eeg(SUB, NAP)
        label_s = load_label_raw(SUB, NAP)
        bad_s = load_bad_ch(SUB, NAP)
        num_epochs = len(label_s) 
    # preprocess for eeg
        ch_names = eeg_s.ch_names
        data_preprocessed_eeg = pd.DataFrame(0.0, index=range(num_epochs * len(ch_names)), columns=range(4))
        data_preprocessed_eeg.columns = ['data', 'channel_name', 'label', 'subject']
        data_preprocessed_eeg = data_preprocessed_eeg.astype({'data': 'object', 'channel_name':'str', 'label': 'int32', 'subject':'str'})
    #data_preprocessed = np.zeros((num_epochs * num_ch, 1)) # data, ch_ name,  label, subject
        data_preprocessed_eeg['subject'] = SUB
        tmp = eeg_s.to_data_frame()
        for i in range(len(ch_names)):
            if ch_names[i] in bad_s:  
                data_preprocessed_eeg.iloc[i * num_epochs : i * num_epochs + num_epochs, 1] = ch_names[i]
                data_preprocessed_eeg.iloc[i * num_epochs : i * num_epochs + num_epochs, 0] = 'nan'
                data_preprocessed_eeg.iloc[i * num_epochs : i * num_epochs + num_epochs, 2] = label_s[:,1]
                continue
            t = tmp[ch_names[i]].values
            data_preprocessed_eeg.iloc[i * num_epochs : i * num_epochs + num_epochs, 1] = ch_names[i]
            data_preprocessed_eeg.iloc[i * num_epochs : i * num_epochs + num_epochs, 2] = label_s[:,1]
            for j in range(num_epochs):
                id = i * num_epochs + j 
                if label_s[j, 2] == 1:
                    data_preprocessed_eeg.iloc[id, 0] = 'nan'
                    continue
                d = t[j * len_epochs * sampling : j * len_epochs * sampling + len_epochs * sampling]
                d = np.expand_dims(d, axis =1)
                data_preprocessed_eeg.iloc[id, 0] = d
            
        #preprocess for eog
        ch_names = eog_s.ch_names
        data_preprocessed_eog = pd.DataFrame(0.0, index=range(num_epochs * len(ch_names)), columns=range(4))
        data_preprocessed_eog.columns = ['data', 'channel_name', 'label', 'subject']
        data_preprocessed_eog = data_preprocessed_eog.astype({'data': 'object', 'channel_name':'str', 'label': 'int32', 'subject':'str'})
        #data_preprocessed = np.zeros((num_epochs * num_ch, 1)) # data, ch_ name,  label, subject
        data_preprocessed_eog['subject'] = SUB
        tmp = eog_s.to_data_frame()
        for i in range(len(ch_names)):
            if ch_names[i] in bad_s:  
                data_preprocessed_eog.iloc[i * num_epochs : i * num_epochs + num_epochs, 1] = ch_names[i]
                data_preprocessed_eog.iloc[i * num_epochs : i * num_epochs + num_epochs, 0] = 'nan'
                data_preprocessed_eog.iloc[i * num_epochs : i * num_epochs + num_epochs, 2] = label_s[:,1]
                continue
            t = tmp[ch_names[i]].values
            data_preprocessed_eog.iloc[i * num_epochs : i * num_epochs + num_epochs, 1] = ch_names[i]
            data_preprocessed_eog.iloc[i * num_epochs : i * num_epochs + num_epochs, 2] = label_s[:,1]
            for j in range(num_epochs):
                id = i * num_epochs + j 
                if label_s[j, 2] == 1:
                    data_preprocessed_eog.iloc[id, 0] = 'nan'
                    continue
                d = t[j * len_epochs * sampling : j * len_epochs * sampling + len_epochs * sampling]
                d = np.expand_dims(d, axis =1)
                data_preprocessed_eog.iloc[id, 0] = d
                
       # preprocess for eog and ecg 
        ch_names = emg_s.ch_names
        data_preprocessed_emg = pd.DataFrame(0.0, index=range(num_epochs * len(ch_names)), columns=range(4))
        data_preprocessed_emg.columns = ['data', 'channel_name', 'label', 'subject']
        data_preprocessed_emg = data_preprocessed_emg.astype({'data': 'object', 'channel_name':'str', 'label': 'int32', 'subject':'str'})
        #data_preprocessed = np.zeros((num_epochs * num_ch, 1)) # data, ch_ name,  label, subject
        data_preprocessed_emg['subject'] = SUB
        tmp = emg_s.to_data_frame()
        for i in range(len(ch_names)):
            if ch_names[i] in bad_s:  
                data_preprocessed_emg.iloc[i * num_epochs : i * num_epochs + num_epochs, 1] = ch_names[i]
                data_preprocessed_emg.iloc[i * num_epochs : i * num_epochs + num_epochs, 0] = 'nan'
                data_preprocessed_emg.iloc[i * num_epochs : i * num_epochs + num_epochs, 2] = label_s[:,1]
                continue
            t = tmp[ch_names[i]].values
            data_preprocessed_emg.iloc[i * num_epochs : i * num_epochs + num_epochs, 1] = ch_names[i]
            data_preprocessed_emg.iloc[i * num_epochs : i * num_epochs + num_epochs, 2] = label_s[:,1]
            for j in range(num_epochs):
                id = i * num_epochs + j 
                if label_s[j, 2] == 1:
                    data_preprocessed_emg.iloc[id, 0] = 'nan'
                    continue
                d = t[j * len_epochs * sampling : j * len_epochs * sampling + len_epochs * sampling]
                d = np.expand_dims(d, axis =1)
                data_preprocessed_emg.iloc[id, 0] = d         
        
        # append data for each subject 
        data_preprocessed = data_preprocessed_eeg.append(data_preprocessed_eog)
        data_preprocessed = data_preprocessed.append(data_preprocessed_emg)
    data_final = data_final.append(data_preprocessed)

sldcd01_nap2
Extracting parameters from /Volumes/CLPS_Watanabe_Lab/users/Zhiyan/SleepDecoding/Experiment/sbj/sldcd01/nap2/EEG/raw/sldcd01nap2_reref_01100.vhdr...
Setting channel info structure...
Reading 0 ... 2711898  =      0.000 ...  5423.796 secs...


<ipython-input-16-2decfd8c57d9>:3: RuntimeWarning: No coordinate information found for channels ['EMG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(fn_path, eog = ('HEOG', 'VEOG'), preload=True)


DigMontage is a superset of info. 3 in DigMontage will be ignored. The ignored channels are: {'EMG', 'VEOG', 'HEOG'}


<ipython-input-16-2decfd8c57d9>:3: DeprecationWarning: The `montage` parameter from `read_raw_brainvision` is deprecated and will be removed  in version 0.20. Use  raw.set_montage(montage) instead.
  raw = mne.io.read_raw_brainvision(fn_path, eog = ('HEOG', 'VEOG'), preload=True)


Converting "time" to "<class 'numpy.int64'>"...
Converting "time" to "<class 'numpy.int64'>"...
Converting "time" to "<class 'numpy.int64'>"...
sldcd03_nap2
Extracting parameters from /Volumes/CLPS_Watanabe_Lab/users/Zhiyan/SleepDecoding/Experiment/sbj/sldcd03/nap2/EEG/raw/sldcd03nap2_reref_01100.vhdr...
Setting channel info structure...
Reading 0 ... 2714879  =      0.000 ...  5429.758 secs...


...

Converting "time" to "<class 'numpy.int64'>"...
Converting "time" to "<class 'numpy.int64'>"...
Converting "time" to "<class 'numpy.int64'>"...
sldcd06_nap2
Extracting parameters from /Volumes/CLPS_Watanabe_Lab/users/Zhiyan/SleepDecoding/Experiment/sbj/sldcd06/nap2/EEG/raw/sldcd06nap2_reref_01100.vhdr...
Setting channel info structure...
Reading 0 ... 2725948  =      0.000 ...  5451.896 secs...


<ipython-input-16-2decfd8c57d9>:3: RuntimeWarning: No coordinate information found for channels ['EMG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(fn_path, eog = ('HEOG', 'VEOG'), preload=True)


DigMontage is a superset of info. 3 in DigMontage will be ignored. The ignored channels are: {'EMG', 'VEOG', 'HEOG'}


<ipython-input-16-2decfd8c57d9>:3: DeprecationWarning: The `montage` parameter from `read_raw_brainvision` is deprecated and will be removed  in version 0.20. Use  raw.set_montage(montage) instead.
  raw = mne.io.read_raw_brainvision(fn_path, eog = ('HEOG', 'VEOG'), preload=True)


Converting "time" to "<class 'numpy.int64'>"...
Converting "time" to "<class 'numpy.int64'>"...
Converting "time" to "<class 'numpy.int64'>"...


In [19]:
data_final.shape
data_final.to_pickle('processed_sleep.pkl')

In [ ]:
## append data frame to a whole data frame 
